# CONSULTA BD

In [ ]:
import pandas as pd
import os

# Detectar entorno para Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# Definir la ruta base para los archivos en Google Drive
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'

# Cargar el DataFrame de hermanos desde la ruta correcta
df_hermanos = pd.read_csv(os.path.join(RUTA_BASE, "base_datos_hermanos.csv"))

df_hermanos[(df_hermanos['genero'] == 'M') & (df_hermanos['privilegio'] == 'ANC')]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,nombre_completo,genero,privilegio,activo,f_presidente,f_oracion,f_lectura,f_tesoros,f_perlas,f_vida,f_estudio_libro,f_lector_libro,f_discurso_est,f_demos,f_ayudante,observaciones
4,102,Juan Pablo Alfonso,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
13,11,James Cabieles,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
19,13,Cristian Calderón,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
29,58,William Diaz,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
47,3,Cesar Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
49,1,Orlando Hernandez,M,ANC,1,1,1,0,1,1,1,1,1,0,1,1,NaN
68,9,Ricardo Morales,M,ANC,1,1,1,0,1,1,1,1,1,0,0,0,NaN
82,23,Edwin Quintero,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN
86,27,Gustavo Ramirez,M,ANC,1,1,1,0,1,1,1,1,0,0,1,1,NaN
88,17,Juan Carlos Riaño,M,ANC,1,1,1,0,1,1,1,1,1,1,1,1,NaN


# 📥 FASE 1 v2.0: EXTRACTOR WEB ROBUSTO (WOL)

In [9]:
# =============================================================================
#  🕷️ FASE 1 v4.0: EXTRACTOR "CAZADOR" (INFALIBLE)
# =============================================================================
#  DIFERENCIA CLAVE:
#  No confía en la estructura de la página (divs/sections).
#  Busca "palabras clave" en todo el documento HTML.
#  Esto garantiza que encuentre las partes de Maestros sí o sí.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import json
import re
import os
import time

# RUTAS
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

def obtener_html(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(url, headers=headers, timeout=20)
        if response.status_code == 200:
            return BeautifulSoup(response.content, 'html.parser')
    except: pass
    return None

def limpiar_texto(texto):
    if not texto: return ""
    return texto.replace('\u00a0', ' ').strip()

def generar_calendario_cazador():
    print("🌍 INICIANDO EXTRACCIÓN CAZADOR (v4.0)")
    print("=======================================")

    # Fechas Lunes Enero 2026
    fechas = ["2026/1/5", "2026/1/12", "2026/1/19", "2026/1/26"]
    datos_estructurados = []

    for fecha_url in fechas:
        url = f"https://wol.jw.org/es/wol/dt/r4/lp-s/{fecha_url}"
        print(f"\n   📥 Cazando en: {fecha_url} ...")

        soup = obtener_html(url)
        if not soup:
            print("      ❌ Error de descarga. Saltando.")
            continue

        # 1. TÍTULO SEMANA
        h1 = soup.find('h1', id='p1')
        texto_semana = limpiar_texto(h1.get_text()) if h1 else f"Semana {fecha_url}"
        print(f"      📅 {texto_semana}")

        partes = []

        # --- PLATAFORMA ---
        partes.append({"tipo": "Presidente", "titulo": "Presidente de la Reunión"})
        partes.append({"tipo": "Oracion_Inicio", "titulo": "Oración de inicio"})

        # --- CAZANDO TESOROS (Texto libre) ---
        # Buscamos cualquier párrafo o título que tenga "10 mins"
        encontrado_tesoros = False
        for elem in soup.find_all(['p', 'h3', 'h4']):
            txt = limpiar_texto(elem.get_text())
            if "(10 mins.)" in txt and "Perlas" not in txt and "Consejo" not in txt:
                tit = txt.replace("(10 mins.)", "").strip()
                partes.append({"tipo": "Tesoros", "titulo": tit})
                encontrado_tesoros = True
                break

        partes.append({"tipo": "Perlas", "titulo": "Busquemos Perlas Escondidas"})

        # Cazando Lectura (4 mins)
        for elem in soup.find_all(['p', 'h3', 'h4']):
            txt = limpiar_texto(elem.get_text())
            if "(4 mins.)" in txt:
                match = re.search(r'\)\s*(.+)', txt)
                tit = match.group(1).strip() if match else "Lectura de la Biblia"
                partes.append({"tipo": "Lectura", "titulo": tit})
                break

        # --- CAZANDO MAESTROS (Estrategia agresiva) ---
        # Buscamos TODOS los h3 y filtramos por palabras clave
        contador_maestros = 0
        palabras_maestras = ["Empiece", "Haga revisitas", "Haga discípulos", "Explique", "Discurso"]

        all_h3 = soup.find_all('h3')
        for h3 in all_h3:
            txt = limpiar_texto(h3.get_text())

            # Filtros de seguridad (para no confundir títulos de sección)
            if "MAESTROS" in txt.upper() or "Estudio" in txt or "Lectura" in txt: continue

            # Chequeo: ¿Contiene alguna palabra clave?
            es_maestro = False
            for key in palabras_maestras:
                if key in txt:
                    es_maestro = True
                    break

            if es_maestro:
                # Limpieza: Quitamos "(X mins.)" y lo anterior
                # Ejemplo: "4. Empiece conversaciones (3 mins.)" -> "Empiece conversaciones"
                tit_clean = re.split(r'\(\d', txt)[0].strip().strip('.:0123456789')
                # Quitamos números iniciales "4. "

                partes.append({"tipo": "Maestros", "titulo": tit_clean})
                contador_maestros += 1
                print(f"         ✅ Encontrado: {tit_clean}")

        if contador_maestros == 0:
            print("         ⚠️ ALERTA: No encontré partes de Maestros. Revisar web.")

        # --- CAZANDO VIDA CRISTIANA ---
        # Todo h3 que NO sea lo anterior y esté en la segunda mitad del documento
        # (Estrategia simplificada: buscamos por exclusión)
        section_vida = soup.find('div', id='section4')
        if section_vida:
            for h3 in section_vida.find_all('h3'):
                txt = limpiar_texto(h3.get_text())
                if "VIDA CRISTIANA" in txt.upper(): continue
                if any(x in txt for x in ["Cántico", "Canción", "Oración", "Estudio", "Conclusión"]): continue

                tit = re.split(r'\(\d', txt)[0].strip()
                partes.append({"tipo": "NVC", "titulo": tit})

        # FIJOS
        partes.append({"tipo": "Estudio_Libro", "titulo": "Estudio Bíblico de Congregación"})
        partes.append({"tipo": "Lector_Libro", "titulo": "Lectura del Estudio Bíblico"})
        partes.append({"tipo": "Oracion_Final", "titulo": "Oración de conclusión"})

        datos_estructurados.append({"semana": texto_semana, "partes": partes})

    with open(FILE_JSON, 'w', encoding='utf-8') as f:
        json.dump(datos_estructurados, f, indent=4, ensure_ascii=False)

    print("\n💾 ¡MAPA COMPLETADO! Ahora el Registrador tendrá todas las partes.")

if __name__ == "__main__":
    generar_calendario_cazador()

🌍 INICIANDO EXTRACCIÓN CAZADOR (v4.0)

   📥 Cazando en: 2026/1/5 ...
      📅 5-11 DE ENERO
         ✅ Encontrado:  Empiece conversaciones
         ✅ Encontrado:  Haga revisitas
         ✅ Encontrado:  Discurso

   📥 Cazando en: 2026/1/12 ...
      📅 12-18 DE ENERO
         ✅ Encontrado:  Empiece conversaciones
         ✅ Encontrado:  Empiece conversaciones
         ✅ Encontrado:  Haga revisitas
         ✅ Encontrado:  Discurso

   📥 Cazando en: 2026/1/19 ...
      📅 19-25 DE ENERO
         ✅ Encontrado:  Empiece conversaciones
         ✅ Encontrado:  Haga revisitas
         ✅ Encontrado:  Explique sus creencias

   📥 Cazando en: 2026/1/26 ...
      📅 26 DE ENERO A 1 DE FEBRERO
         ✅ Encontrado:  Empiece conversaciones
         ✅ Encontrado:  Haga revisitas
         ✅ Encontrado:  Haga discípulos
         ✅ Encontrado:  Haga discípulos

💾 ¡MAPA COMPLETADO! Ahora el Registrador tendrá todas las partes.


# 🚀 FASE 2: Fusión y Generación del Formato

In [4]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 1: EXTRACCIÓN HÍBRIDA (Excel + JSON)
# =============================================================================
#  RAMA: fix-fase1
#  CAMBIOS:
#  1. Genera Excel (Legacy) para respaldo.
#  2. Genera JSON (Nuevo) para el motor de asignación automática.
#  3. Soporta Enero 2026 correctamente.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
import json
import time
import os

# Detectar entorno
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

# CONFIGURACIÓN
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
RUTA_EXCEL = os.path.join(RUTA_BASE, 'Programación VMC_Septiembre-2025-2026.xlsx')
RUTA_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')

COLUMNAS_ORDENADAS = [
    'Semana', 'Libro', 'Canción Inicial', 'Tesoros de la Biblia',
    'Segunda Canción', 'Tercera Canción',
    'Maestros Título 1', 'Maestros Título 2', 'Maestros Título 3', 'Maestros Título 4',
    'NVC Título 1', 'NVC Título 2', 'NVC Título 3',
    'Info Lectura Biblia', 'Info Estudio Libro'
]

def obtener_fecha_interactiva():
    print("📅 CONFIGURACIÓN DE BÚSQUEDA")
    try:
        anio = int(input("👉 AÑO (ej. 2026): "))
        mes = int(input("👉 MES (1-12): "))
        return anio, mes
    except: return None, None

def generar_urls_dinamicas(anio, mes):
    print(f"\n🔍 Calculando semanas para: {mes}/{anio}...")
    urls = []
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    for week in c.monthdatescalendar(anio, mes):
        if week[0].month == mes:
            urls.append(f"https://wol.jw.org/es/wol/dt/r4/lp-s/{week[0].year}/{week[0].month:02d}/{week[0].day:02d}")
    return urls

def limpiar_texto_estudio(texto):
    if not texto: return ""
    libros = ['lfb', 'bt', 'lmd', 'lvs', 'cf', 'rr', 'ia', 'jr']
    for l in libros: texto = re.sub(rf"\b{l}(?=[a-zA-Z0-9])", f"{l} ", texto, flags=re.IGNORECASE)
    return texto

def extraer_informacion(url):
    try:
        soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).content, 'html.parser')
        info = {col: "" for col in COLUMNAS_ORDENADAS}

        # Básicos
        info['Semana'] = soup.find('h1', id='p1').get_text(strip=True) if soup.find('h1', id='p1') else "N/A"
        if soup.find('h2', id='p2'): info['Libro'] = soup.find('h2', id='p2').get_text(strip=True)

        # Tesoros (Lógica Refinada S-38: Discurso de 10 mins)
        for elem in soup.find_all(lambda t: t.name in ['h3','p'] and "(10 mins.)" in t.text):
            if "Perlas" not in elem.text:
                info['Tesoros de la Biblia'] = elem.get_text(strip=True).replace("(10 mins.)", "").strip()
                break

        # Canciones
        canciones = [f"Canción {re.search(r'(\d+)', h3.text).group(1)}" for h3 in soup.find_all('h3') if "Canción" in h3.text and re.search(r'(\d+)', h3.text)]
        if len(canciones) > 0: info['Canción Inicial'] = canciones[0]
        if len(canciones) > 1: info['Segunda Canción'] = canciones[1]
        if len(canciones) > 2: info['Tercera Canción'] = canciones[2]

        # Lectura y Estudio
        for p in soup.find_all('p'):
            if "(4 mins.)" in p.text and not info['Info Lectura Biblia']:
                info['Info Lectura Biblia'] = re.search(r'\((?:4|3)\s+mins?.*?\)\s*(.+)', p.text).group(1).strip()
            if "(30 mins.)" in p.text and not info['Info Estudio Libro']:
                info['Info Estudio Libro'] = limpiar_texto_estudio(re.search(r'\(30\s+mins?.*?\)\s*(.+)', p.text).group(1).strip())

        # Secciones Maestros y Vida
        seccion, titulos_maestros, nvc_titulos = None, [], []
        for h in soup.find_all(['h2', 'h3']):
            txt = h.get_text(strip=True)
            if h.name == 'h2':
                if "MEJORES MAESTROS" in txt.upper(): seccion = "MAESTROS"
                elif "VIDA CRISTIANA" in txt.upper(): seccion = "VIDA"
            elif h.name == 'h3' and seccion:
                if any(x in txt for x in ["Canción", "Conclusión", "Artículo"]): continue
                if "Artículo de estudio" in txt: break

                if seccion == "MAESTROS": titulos_maestros.append(txt)
                elif seccion == "VIDA" and "Estudio bíblico" not in txt: nvc_titulos.append(txt)

        for i, t in enumerate(titulos_maestros[:4]): info[f'Maestros Título {i+1}'] = t
        for i, t in enumerate(nvc_titulos[:3]): info[f'NVC Título {i+1}'] = t
        info['NVC Título 3'] = "Estudio bíblico de la congregación" # Default

        return info
    except Exception as e:
        print(f"❌ Error {url}: {e}")
        return None

def guardar_excel_limpio(datos):
    print(f"\n💾 (LEGACY) Guardando Excel: {os.path.basename(RUTA_EXCEL)}...")
    try:
        book = load_workbook(RUTA_EXCEL) if os.path.exists(RUTA_EXCEL) else pd.ExcelWriter(RUTA_EXCEL, engine='openpyxl').book
        if 'Info-reunion' not in book.sheetnames: book.create_sheet('Info-reunion')
        ws = book['Info-reunion']

        if ws.max_row > 1: ws.delete_rows(2, amount=ws.max_row-1) # Limpiar

        for r_idx, row in enumerate(dataframe_to_rows(pd.DataFrame(datos, columns=COLUMNAS_ORDENADAS), index=False, header=False), 1):
            for c_idx, val in enumerate(row, 1): ws.cell(row=r_idx+1, column=c_idx, value=val)

        book.save(RUTA_EXCEL)
        print("✅ Excel actualizado.")
    except Exception as e: print(f"❌ Error Excel: {e}")

# --- AQUÍ ESTÁ LA NUEVA FUNCIÓN JSON ---
def guardar_json_reuniones(datos):
    print(f"\n💾 (NUEVO) Generando JSON para Motor de Asignación...")
    data_estructurada = []

    for d in datos:
        # Aquí aplicamos lógica básica basada en S-38 para definir requisitos
        semana_obj = {
            "semana": d['Semana'],
            "partes": [
                {"tipo": "Presidente", "titulo": "Presidencia", "requiere": "ANC", "tiempo": 0},
                {"tipo": "Oracion_Inicio", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5},
                {"tipo": "Tesoros", "titulo": d['Tesoros de la Biblia'], "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Perlas", "titulo": "Perlas Escondidas", "requiere": "ANC_SM", "tiempo": 10},
                {"tipo": "Lectura", "titulo": d['Info Lectura Biblia'], "requiere": "EST_VARON", "tiempo": 4}
            ]
        }

        # Maestros (Dinámico)
        for i in range(1, 5):
            titulo = d.get(f'Maestros Título {i}')
            if titulo:
                # Análisis simple de texto para inferir si es Discurso (Solo varones)
                req = "ESTUDIANTE"
                if "Discurso" in titulo: req = "EST_VARON"
                semana_obj['partes'].append({"tipo": "Maestros", "titulo": titulo, "requiere": req, "tiempo": 5})

        # Vida Cristiana
        for i in range(1, 3):
            titulo = d.get(f'NVC Título {i}')
            if titulo:
                semana_obj['partes'].append({"tipo": "NVC", "titulo": titulo, "requiere": "ANC_SM", "tiempo": 15})

        semana_obj['partes'].extend([
            {"tipo": "Estudio_Libro", "titulo": "Estudio Bíblico", "requiere": "ANC_SM", "tiempo": 30},
            {"tipo": "Lector_Libro", "titulo": "Lectura EBC", "requiere": "H_BAUTIZADO", "tiempo": 0},
            {"tipo": "Oracion_Final", "titulo": "Oración", "requiere": "H_BAUTIZADO", "tiempo": 5}
        ])

        data_estructurada.append(semana_obj)

    with open(RUTA_JSON, 'w', encoding='utf-8') as f:
        json.dump(data_estructurada, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON guardado en: {os.path.basename(RUTA_JSON)}")

# EJECUCIÓN
if __name__ == "__main__":
    anio, mes = obtener_fecha_interactiva()
    if anio:
        datos = [extraer_informacion(url) for url in generar_urls_dinamicas(anio, mes) if extraer_informacion(url)]
        if datos:
            guardar_excel_limpio(datos)      # Mantenemos el puente viejo
            guardar_json_reuniones(datos)    # Construimos el puente nuevo
            print("\n🎉 FASE 1 COMPLETADA (HÍBRIDA).")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📅 CONFIGURACIÓN DE BÚSQUEDA


# Utilidad: Crear archivo de Ausencias


In [2]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR AUSENCIAS (CORREGIDO)
# =============================================================================
#  Crea el archivo ausencias.csv manejando la conexión a Google Drive.
# =============================================================================

import pandas as pd
import os
from google.colab import drive # 1. Importamos la librería de conexión

def inicializar_ausencias():
    print("🚑 INICIALIZANDO SISTEMA DE AUSENCIAS")
    print("-------------------------------------")

    # 2. VERIFICACIÓN DE CONEXIÓN (El paso clave que faltaba)
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # 3. RUTAS
    # Verifica que esta ruta sea la correcta (sin espacios extraños)
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    FILE_AUSENCIAS = os.path.join(RUTA_BASE, 'ausencias.csv')

    # Validar que la carpeta madre exista
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR: No encuentro la carpeta del proyecto en:")
        print(f"   {RUTA_BASE}")
        return

    # 4. CREACIÓN DEL ARCHIVO
    if not os.path.exists(FILE_AUSENCIAS):
        # Columnas estándar para gestionar vacaciones
        cols = ['id_hermano', 'nombre', 'fecha_inicio', 'fecha_fin', 'motivo']

        df = pd.DataFrame(columns=cols)
        df.to_csv(FILE_AUSENCIAS, index=False, encoding='utf-8')

        print(f"\n✅ ¡ÉXITO! Archivo creado: {os.path.basename(FILE_AUSENCIAS)}")
        print("   Ahora puedes registrar las fechas en que los hermanos no estarán.")
    else:
        print(f"\n⚠️ El archivo ya existe en: {os.path.basename(FILE_AUSENCIAS)}")
        print("   No se ha borrado nada.")

if __name__ == "__main__":
    inicializar_ausencias()

🚑 INICIALIZANDO SISTEMA DE AUSENCIAS
-------------------------------------
✅ Google Drive ya está conectado.

✅ ¡ÉXITO! Archivo creado: ausencias.csv
   Ahora puedes registrar las fechas en que los hermanos no estarán.


# Uilidad 2: Agregar hermano nuevo

In [ ]:
# =============================================================================
#  🛠️ GESTIÓN DE HERMANOS: AGREGAR NUEVO (SIN ROMPER IDs)
# =============================================================================

import pandas as pd
import os

RUTA_BD = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/base_datos_hermanos.csv'

def agregar_hermano_manual():
    print("👤 INSCRIPCIÓN DE NUEVO HERMANO")
    print("-------------------------------")

    # 1. Cargar base de datos actual
    if not os.path.exists(RUTA_BD):
        print("❌ Error: No existe la base de datos.")
        return

    df = pd.read_csv(RUTA_BD)

    # 2. Calcular ID Automático (Max ID + 1) para integridad
    # Si la tabla está vacía, empieza en 1. Si no, busca el máximo.
    nuevo_id = 1
    if not df.empty:
        nuevo_id = df['id'].max() + 1

    print(f"🆔 ID Asignado Automáticamente: {nuevo_id}")

    # 3. Pedir datos
    nombre = input("Nombre Completo: ").strip().title()
    genero = input("Género (M/F): ").strip().upper()
    privilegio = input("Privilegio (PUB/SM/ANC): ").strip().upper()

    # Capacidades básicas (Preguntas rápidas)
    print("\n--- Capacidades (S/N) ---")
    lectura = 1 if input("¿Hace Lectura de Biblia? ").upper() == 'S' else 0
    demos = 1 if input("¿Hace Demostraciones/Ayudante? ").upper() == 'S' else 0
    discurso = 1 if input("¿Hace Discursos Estudiantiles? ").upper() == 'S' else 0

    # 4. Crear el diccionario con la estructura EXACTA
    nuevo_hermano = {
        'id': nuevo_id,
        'nombre_completo': nombre,
        'genero': genero,
        'privilegio': privilegio,
        'activo': 1,
        # Banderas
        'f_presidente': 0, 'f_oracion': 0,
        'f_lectura': lectura,
        'f_tesoros': 0, 'f_perlas': 0, 'f_vida': 0,
        'f_estudio_libro': 0, 'f_lector_libro': 0,
        'f_discurso_est': discurso,
        'f_demos': demos,
        'f_ayudante': demos, # Generalmente si hace demos, es ayudante
        'observaciones': "Ingreso manual"
    }

    # 5. Guardar
    # Convertimos el diccionario a DataFrame y lo concatenamos
    nuevo_df = pd.DataFrame([nuevo_hermano])

    # Alineamos columnas (para evitar desorden si faltan claves)
    # Usamos las columnas del df original como molde
    for col in df.columns:
        if col not in nuevo_df.columns:
            nuevo_df[col] = 0 # Rellenar con 0 lo que no preguntamos

    df_final = pd.concat([df, nuevo_df], ignore_index=True)
    df_final.to_csv(RUTA_BD, index=False, encoding='utf-8-sig')

    print(f"\n✅ ¡Listo! {nombre} ha sido inscrito con el ID {nuevo_id}.")

# Para usarlo, simplemente ejecuta:
# agregar_hermano_manual()

# Previo: Inicializar la Memoria (Historial Asginaciones)

In [1]:
# =============================================================================
#  🛠️ UTILIDAD: INICIALIZAR HISTORIAL (CON AUTO-CONEXIÓN DRIVE)
# =============================================================================

import pandas as pd
import os
from google.colab import drive # Importamos la librería de Drive

def inicializar_historial():
    # --- PASO 1: VERIFICAR CONEXIÓN A DRIVE ---
    if not os.path.exists('/content/drive'):
        print("🔌 Conectando a Google Drive...")
        drive.mount('/content/drive')
    else:
        print("✅ Google Drive ya está conectado.")

    # --- PASO 2: CONFIGURACIÓN DE RUTA ---
    # Verifica que esta ruta sea EXACTAMENTE la de tu carpeta
    RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
    RUTA_HISTORIAL = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')

    # Verificar si la carpeta existe antes de intentar guardar
    if not os.path.exists(RUTA_BASE):
        print(f"❌ ERROR CRÍTICO: No encuentro la carpeta en Drive.")
        print(f"   Buscaba: {RUTA_BASE}")
        print("   👉 Verifica si el nombre de la carpeta tiene espacios o tildes diferentes.")
        return

    print(f"\n📂 Verificando historial en: {os.path.basename(RUTA_HISTORIAL)}")

    if os.path.exists(RUTA_HISTORIAL):
        print(f"⚠️ El archivo ya existe. No se ha borrado nada por seguridad.")
    else:
        columnas_maestras = [
            'fecha_asignacion', 'semana_texto', 'id_hermano',
            'nombre_hermano', 'tipo_asignacion', 'sala'
        ]

        df_vacio = pd.DataFrame(columns=columnas_maestras)
        df_vacio.to_csv(RUTA_HISTORIAL, index=False, encoding='utf-8')
        print(f"✅ ¡ÉXITO! Archivo de historial creado y listo.")

if __name__ == "__main__":
    inicializar_historial()

🔌 Conectando a Google Drive...
Mounted at /content/drive

📂 Verificando historial en: historial_asignaciones.csv
✅ ¡ÉXITO! Archivo de historial creado y listo.


# 🟩 Código Fase C: El Motor de Asignación v3.0

In [ ]:
# =============================================================================
#  🧠 MOTOR DE ASIGNACIÓN VMC - V3.1 (DUPLICADOS Y ALERTAS INTELIGENTES)
# =============================================================================
#  CAMBIOS:
#  1. Bloqueo Estricto: Estudiantes no pueden repetir parte en la misma semana.
#  2. Bloqueo Suave: Ancianos/Siervos pueden repetir si no hay nadie más.
#  3. Alertas: El sistema avisa si tuvo que duplicar asignación por falta de personal.
#  4. Excepción Presidente: Se marca diferente si el Presidente cubre una parte.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime, timedelta

# --- CONFIGURACIÓN DE RUTAS ---
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HIST = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_AUSENCIAS = os.path.join(RUTA_BASE, 'ausencias.csv')
FILE_JSON_IN = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')
FILE_JSON_OUT = os.path.join(RUTA_BASE, 'programa_asignado_2026.json')

# --- DEFINICIONES DE NEGOCIO ---
REGLAS_DB = {
    "Presidente": "f_presidente",
    "Oracion_Inicio": "f_oracion", "Oracion_Final": "f_oracion",
    "Tesoros": "f_tesoros", "Perlas": "f_perlas",
    "Lectura": "f_lectura", "Maestros": "f_demos", "Ayudante": "f_ayudante",
    "NVC": "f_vida",
    "Estudio_Libro": "f_estudio_libro", "Lector_Libro": "f_lector_libro"
}

def cargar_datos():
    if not os.path.exists(FILE_DB) or not os.path.exists(FILE_JSON_IN): return None, None, None, None
    df_h = pd.read_csv(FILE_DB)

    if os.path.exists(FILE_HIST):
        df_hist = pd.read_csv(FILE_HIST)
        df_hist['fecha_asignacion'] = pd.to_datetime(df_hist['fecha_asignacion'], errors='coerce')
    else:
        df_hist = pd.DataFrame(columns=['fecha_asignacion', 'id_hermano', 'tipo_asignacion'])

    if os.path.exists(FILE_AUSENCIAS):
        df_aus = pd.read_csv(FILE_AUSENCIAS)
        df_aus['fecha_inicio'] = pd.to_datetime(df_aus['fecha_inicio'], errors='coerce')
        df_aus['fecha_fin'] = pd.to_datetime(df_aus['fecha_fin'], errors='coerce')
    else:
        df_aus = pd.DataFrame(columns=['id_hermano', 'fecha_inicio', 'fecha_fin'])

    with open(FILE_JSON_IN, 'r', encoding='utf-8') as f: data_j = json.load(f)
    return df_h, df_hist, df_aus, data_j

def esta_ausente(id_h, fecha, df_aus):
    if df_aus.empty: return False
    vacs = df_aus[df_aus['id_hermano'] == id_h]
    for _, v in vacs.iterrows():
        if v['fecha_inicio'] <= fecha <= v['fecha_fin']: return True
    return False

def buscar_candidato(tipo_parte, df_h, df_hist, df_aus, excluidos_ids, fecha_reunion, genero_forzado=None, permitir_duplicado=False, id_presidente=None):
    """
    Busca candidato con lógica de intentos:
    - Intento 1: Busca libres (sin duplicar).
    - Intento 2: Si permitir_duplicado=True y falla el intento 1, busca entre los ocupados (emergencia).
    """
    col_req = REGLAS_DB.get(tipo_parte)
    if not col_req: return None, None # Candidato, Mensaje_Alerta

    # 1. Base Candidatos (Activos y Capaces)
    base = df_h[(df_h['activo'] == 1) & (df_h[col_req] == 1)].copy()
    if genero_forzado: base = base[base['genero'] == genero_forzado]

    # Filtro de Ausencias (Siempre aplica)
    base = base[~base['id'].apply(lambda x: esta_ausente(x, pd.to_datetime(fecha_reunion), df_aus))]

    if base.empty: return None, "SIN CANDIDATOS EN BASE"

    # --- INTENTO 1: SIN DUPLICADOS (Ideal) ---
    candidatos_libres = base[~base['id'].isin(excluidos_ids)]

    if not candidatos_libres.empty:
        return _elegir_por_score(candidatos_libres, df_hist, tipo_parte, fecha_reunion), None

    # --- INTENTO 2: CON DUPLICADOS (Emergencia) ---
    if permitir_duplicado:
        # Buscamos entre TODOS (incluso los ocupados), pero penalizamos score
        # Nota: Aún así respetamos Ausencias y Género
        candidato_emergencia = _elegir_por_score(base, df_hist, tipo_parte, fecha_reunion)

        if candidato_emergencia is not None:
            cid = candidato_emergencia['id']
            msg = "⚠️ REPETIDO"
            if id_presidente and cid == id_presidente:
                msg = "🛡️ COBERTURA (Presidente)"
            elif cid in excluidos_ids:
                msg = "⚠️ DOBLE ASIGNACIÓN"
            return candidato_emergencia, msg

    return None, "SIN DISPONIBLES"

def _elegir_por_score(candidatos, df_hist, tipo_parte, fecha_ref):
    scores = []
    fecha_ref = pd.to_datetime(fecha_ref)

    for _, h in candidatos.iterrows():
        h_id = h['id']

        # Historial específico
        hist_esp = df_hist[(df_hist['id_hermano'] == h_id) & (df_hist['tipo_asignacion'] == tipo_parte)]
        fecha_score = hist_esp['fecha_asignacion'].max() if not hist_esp.empty else datetime(2000, 1, 1)

        # Penalización por saturación reciente (opcional, aquí simplificado)
        scores.append({'id': h_id, 'nombre': h['nombre_completo'], 'genero': h['genero'], 'score_fecha': fecha_score})

    df_scores = pd.DataFrame(scores).sample(frac=1, random_state=42).sort_values('score_fecha', ascending=True)
    return df_scores.iloc[0] if not df_scores.empty else None

def ejecutar_asignaciones():
    print("🚀 MOTOR VMC v3.1 - GESTIÓN DE DUPLICADOS")
    print("=========================================")
    print("1. Todo | 2. Solo No Estudiantes | 3. Solo Estudiantes")
    op = input("👉 Opción: ")

    cat_est = ['Lectura', 'Maestros']
    cat_no_est = ['Presidente', 'Oracion_Inicio', 'Oracion_Final', 'Tesoros', 'Perlas', 'NVC', 'Estudio_Libro', 'Lector_Libro']

    procesar = []
    if op == '1': procesar = cat_est + cat_no_est
    elif op == '2': procesar = cat_no_est
    elif op == '3': procesar = cat_est
    else: return

    df_h, df_hist, df_aus, data_json = cargar_datos()
    if df_h is None: return "❌ Error carga"

    nuevas = []
    prog_final = []
    fecha_base = datetime(2026, 1, 5)

    for idx_sem, semana in enumerate(data_json):
        fecha_reunion = fecha_base + timedelta(weeks=idx_sem)
        print(f"\n📅 {semana['semana']}")

        sem_out = semana.copy(); sem_out['asignaciones'] = []
        ocupados = []
        id_presidente = None # Rastreamos quién es presidente esta semana

        # --- FASE 1: Identificar Presidente (si ya se asignó o se va a asignar) ---
        # Si asignamos todo, el presidente es el primero. Si asignamos parcial,
        # deberíamos leer quién es, pero por ahora asumimos flujo secuencial.

        for parte in semana['partes']:
            tipo = parte['tipo']; titulo = parte['titulo']

            if tipo not in procesar:
                # Si saltamos la parte, intentamos ver si era el presidente para guardarlo en memoria
                # (Mejora futura: leer del historial si ya estaba asignado)
                p_out = parte.copy(); p_out['asignado_a'] = "MANUAL / NO TOCAR"
                sem_out['asignaciones'].append(p_out)
                continue

            # CONFIGURACIÓN DE REGLAS POR TIPO
            es_estudiante = tipo in ['Lectura', 'Maestros', 'Ayudante']
            permitir_doble = not es_estudiante # Estudiantes NO repiten (Strict). Otros SI (Soft).

            # Parametros especiales
            gen_req = 'M' if (tipo == 'Lectura' or (tipo == 'Maestros' and 'Discurso' in titulo)) else None

            # --- SELECCIÓN MAESTROS (Con Parejas) ---
            if tipo == "Maestros":
                # 1. Estudiante
                cand, alerta = buscar_candidato(tipo, df_h, df_hist, df_aus, ocupados, fecha_reunion, gen_req, permitir_doble, id_presidente)

                if cand is not None:
                    est_nom = cand['nombre']; est_id = cand['id']; est_gen = cand['genero']
                    ocupados.append(est_id)
                    nuevas.append({'fecha_asignacion': fecha_reunion, 'semana_texto': semana['semana'], 'id_hermano': est_id, 'nombre_hermano': est_nom, 'tipo_asignacion': tipo, 'sala': 'Principal'})
                    df_hist = pd.concat([df_hist, pd.DataFrame([{'fecha_asignacion': fecha_reunion, 'id_hermano': est_id, 'tipo_asignacion': tipo}])], ignore_index=True)

                    # 2. Ayudante
                    ayu_nom = ""
                    if "Discurso" not in titulo:
                        cand_ayu, alerta_ayu = buscar_candidato("Ayudante", df_h, df_hist, df_aus, ocupados, fecha_reunion, est_gen, permitir_doble, id_presidente)
                        if cand_ayu:
                            ayu_nom = cand_ayu['nombre']
                            ocupados.append(cand_ayu['id'])
                            nuevas.append({'fecha_asignacion': fecha_reunion, 'semana_texto': semana['semana'], 'id_hermano': cand_ayu['id'], 'nombre_hermano': ayu_nom, 'tipo_asignacion': 'Ayudante', 'sala': 'Principal'})
                            print(f"   ✅ {tipo:<10} -> {est_nom} + {ayu_nom}")
                        else:
                            ayu_nom = "FALTA AYUDANTE"
                            print(f"   ⚠️ {tipo:<10} -> {est_nom} + ❌ (Sin Ayudante)")
                    else:
                        print(f"   ✅ Discurso   -> {est_nom}")

                    p_out = parte.copy(); p_out['asignado_a'] = est_nom
                    if ayu_nom: p_out['ayudante'] = ayu_nom
                    sem_out['asignaciones'].append(p_out)
                else:
                    print(f"   🛑 {tipo:<10} -> SIN ESTUDIANTE ({alerta})")
                    p_out = parte.copy(); p_out['asignado_a'] = "PENDIENTE"
                    sem_out['asignaciones'].append(p_out)

            # --- SELECCIÓN RESTO (Plataforma) ---
            else:
                cand, alerta = buscar_candidato(tipo, df_h, df_hist, df_aus, ocupados, fecha_reunion, gen_req, permitir_doble, id_presidente)

                nom = "PENDIENTE"
                if cand is not None:
                    nom = cand['nombre']; cid = cand['id']
                    ocupados.append(cid)
                    if tipo == 'Presidente': id_presidente = cid # Guardamos al presi

                    # Logica de visualización de alerta
                    prefix = "✅"
                    suffix = ""
                    if alerta:
                        prefix = "⚠️" if "REPETIDO" in alerta else "🛡️"
                        suffix = f" [{alerta}]"

                    nuevas.append({'fecha_asignacion': fecha_reunion, 'semana_texto': semana['semana'], 'id_hermano': cid, 'nombre_hermano': nom, 'tipo_asignacion': tipo, 'sala': 'Principal'})
                    df_hist = pd.concat([df_hist, pd.DataFrame([{'fecha_asignacion': fecha_reunion, 'id_hermano': cid, 'tipo_asignacion': tipo}])], ignore_index=True)
                    print(f"   {prefix} {tipo:<10} -> {nom}{suffix}")
                else:
                    print(f"   🛑 {tipo:<10} -> SIN CANDIDATO ({alerta})")

                p_out = parte.copy(); p_out['asignado_a'] = nom
                sem_out['asignaciones'].append(p_out)

        prog_final.append(sem_out)

    with open(FILE_JSON_OUT, 'w', encoding='utf-8') as f: json.dump(prog_final, f, indent=4, ensure_ascii=False)

    if nuevas:
        pd.DataFrame(nuevas).to_csv(os.path.join(RUTA_BASE, 'historial_temp_revision.csv'), index=False, encoding='utf-8')
        print(f"\n💾 Guardado: programa_asignado_2026.json")

if __name__ == "__main__":
    ejecutar_asignaciones()

🚀 MOTOR DE ASIGNACIÓN VMC v2.0

¿Qué deseas asignar hoy?
1. TODO el programa (Completo)
2. Solo partes NO ESTUDIANTILES (Plataforma, Ancianos, NVC)
3. Solo partes ESTUDIANTILES (Seamos Mejores Maestros)


KeyboardInterrupt: Interrupted by user

# Registrador Manual v3.2 (Full Automático)

In [10]:
# =============================================================================
#  📝 REGISTRADOR MANUAL v3.6 (DIAGNÓSTICO + FLUJO SEGURO)
# =============================================================================
#  MEJORAS:
#  1. Muestra la lista de asignaciones DETECTADAS antes de empezar.
#     (Si no ves 'Empiece conversaciones' ahí, es culpa del JSON).
#  2. Asegura que ENTER sirva para "Saltar a la siguiente" sin cerrar.
# =============================================================================

import pandas as pd
import json
import os
from datetime import datetime, timedelta

# RUTAS
RUTA_BASE = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/'
FILE_DB = os.path.join(RUTA_BASE, 'base_datos_hermanos.csv')
FILE_HIST = os.path.join(RUTA_BASE, 'historial_asignaciones.csv')
FILE_JSON = os.path.join(RUTA_BASE, 'data_reuniones_2026.json')
FECHA_BASE_INICIO = datetime(2026, 1, 5)

def buscar_y_seleccionar(mensaje, df_hermanos):
    while True:
        print(f"      ❓ {mensaje}")
        nombre_busq = input("         Escribe Nombre (o ENTER para saltar): ").strip()

        # SI EL USUARIO DA ENTER (VACÍO) -> RETORNAMOS 'None' PERO SEGUIMOS EL FLUJO
        if not nombre_busq: return None, None, None

        candidatos = df_hermanos[df_hermanos['nombre_completo'].str.contains(nombre_busq, case=False, na=False)]

        if candidatos.empty:
            print("         ❌ No encontrado.")
            continue

        lista = candidatos.values.tolist()
        for i, h in enumerate(lista):
            print(f"         [{i}] {h[1]} ({h[2]})")

        sel = input("         👉 Elige #: ")
        if sel.isdigit() and int(sel) < len(lista):
            elegido = lista[int(sel)]
            return elegido[0], elegido[1], elegido[2]
        else:
            print("         ⚠️ Opción inválida.")

def obtener_asignado_actual(df_hist, semana, tipo):
    if df_hist.empty: return None
    filtro = (df_hist['semana_texto'] == semana) & (df_hist['tipo_asignacion'] == tipo)
    resultado = df_hist[filtro]
    if not resultado.empty: return resultado.iloc[-1]['nombre_hermano']
    return None

def registrar_manual_v3_6():
    print("🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)")
    print("======================================")

    if not os.path.exists(FILE_DB) or not os.path.exists(FILE_JSON): return print("❌ Faltan archivos.")
    df_db = pd.read_csv(FILE_DB)
    df_hist = pd.read_csv(FILE_HIST) if os.path.exists(FILE_HIST) else pd.DataFrame(columns=['fecha_asignacion'])
    with open(FILE_JSON, 'r', encoding='utf-8') as f: data_reuniones = json.load(f)

    # 1. SELECCIÓN DE SEMANA
    print("\n📅 ELIGE LA SEMANA:")
    for idx, s in enumerate(data_reuniones):
        print(f"   {idx + 1}. {s['semana']}")

    idx_sem = -1
    while True:
        try:
            val = input("\n👉 Número: ")
            idx_sem = int(val) - 1
            if 0 <= idx_sem < len(data_reuniones): break
        except: pass

    semana_data = data_reuniones[idx_sem]
    semana_texto = semana_data['semana']
    partes_raw = semana_data['partes']
    fecha_lunes = (FECHA_BASE_INICIO + timedelta(weeks=idx_sem)).strftime("%Y-%m-%d")

    # 2. FILTRO
    print("\n   1. Solo Estudiantes | 2. Todo el programa")
    modo = input("   👉 Opción: ")
    tipos_interes = ['Lectura', 'Maestros']
    if modo == '2': tipos_interes += ['Presidente', 'Oracion_Inicio', 'Oracion_Final', 'Tesoros', 'Perlas', 'NVC', 'Estudio_Libro', 'Lector_Libro']

    # --- DIAGNÓSTICO PREVIO ---
    partes_a_procesar = [p for p in partes_raw if p['tipo'] in tipos_interes]

    print(f"\n📋 PLAN DE TRABAJO PARA: {semana_texto}")
    print(f"   Se detectaron {len(partes_a_procesar)} partes en el archivo JSON:")
    for i, p in enumerate(partes_a_procesar):
        print(f"   {i+1}. {p.get('titulo', p['tipo'])}")

    print("\n⚠️ IMPORTANTE: Si faltan partes en la lista de arriba,")
    print("   es porque el JSON está incompleto. Deberás correr la FASE 1 de nuevo.")

    input("\n👉 Presiona ENTER para empezar a registrar...")

    nuevos_registros = []

    # 3. BUCLE DE PREGUNTAS
    for parte in partes_a_procesar:
        tipo = parte['tipo']
        titulo = parte.get('titulo', tipo)

        print(f"\n🔹 {titulo}")

        # Memoria Visual
        actual = obtener_asignado_actual(df_hist, semana_texto, tipo)
        prompt = f"¿Quién?"
        if actual:
            print(f"   ✅ YA TIENE ASIGNADO A: {actual}")
            prompt = f"¿Cambiar a {actual}? (ENTER para saltar)"

        # PREGUNTA PRINCIPAL
        id_h, nom_h, gen_h = buscar_y_seleccionar(prompt, df_db)

        if id_h:
            # Eliminar anterior localmente (si hubiera lógica de borrado complejo)
            # Aquí simplemente agregamos el nuevo al lote de "nuevos_registros"
            nuevos_registros.append({
                'fecha_asignacion': fecha_lunes, 'semana_texto': semana_texto,
                'id_hermano': id_h, 'nombre_hermano': nom_h, 'tipo_asignacion': tipo, 'sala': 'Principal'
            })
            print(f"   💾 Registrado para guardar: {nom_h}")

            # PREGUNTA AYUDANTE
            if tipo == 'Maestros' and "Discurso" not in titulo:
                # Revisar ayudante actual
                actual_ayu = obtener_asignado_actual(df_hist, semana_texto, 'Ayudante') # Simplificado
                prompt_ayu = "¿Quién fue su AYUDANTE?"
                if actual_ayu:
                    print(f"   (Ayudante actual detectado en historial: {actual_ayu})")

                id_a, nom_a, _ = buscar_y_seleccionar(prompt_ayu, df_db)
                if id_a:
                    nuevos_registros.append({
                        'fecha_asignacion': fecha_lunes, 'semana_texto': semana_texto,
                        'id_hermano': id_a, 'nombre_hermano': nom_a, 'tipo_asignacion': 'Ayudante', 'sala': 'Principal'
                    })
                    print(f"   💾 Ayudante registrado: {nom_a}")
        else:
            print("   ⏩ Saltando a la siguiente parte...")

    # 4. GUARDADO FINAL
    if nuevos_registros:
        print("\n" + "="*40)
        print(f"💾 Guardando {len(nuevos_registros)} cambios en el historial...")
        df_nuevos = pd.DataFrame(nuevos_registros)
        df_final = pd.concat([df_hist, df_nuevos], ignore_index=True)
        df_final.to_csv(FILE_HIST, index=False, encoding='utf-8')
        print("✅ Historial Actualizado.")
    else:
        print("\n👋 Finalizado sin cambios nuevos.")

if __name__ == "__main__":
    registrar_manual_v3_6()

🕵️ REGISTRADOR CON DIAGNÓSTICO (v3.6)

📅 ELIGE LA SEMANA:
   1. 5-11 DE ENERO
   2. 12-18 DE ENERO
   3. 19-25 DE ENERO
   4. 26 DE ENERO A 1 DE FEBRERO

👉 Número: 1

   1. Solo Estudiantes | 2. Todo el programa
   👉 Opción: 1

📋 PLAN DE TRABAJO PARA: 5-11 DE ENERO
   Se detectaron 4 partes en el archivo JSON:
   1. Is 19:1-12 (th lección 11).
   2.  Empiece conversaciones
   3.  Haga revisitas
   4.  Discurso

⚠️ IMPORTANTE: Si faltan partes en la lista de arriba,
   es porque el JSON está incompleto. Deberás correr la FASE 1 de nuevo.

👉 Presiona ENTER para empezar a registrar...

🔹 Is 19:1-12 (th lección 11).
   ✅ YA TIENE ASIGNADO A: Duvan Torres
      ❓ ¿Cambiar a Duvan Torres? (ENTER para saltar)
         Escribe Nombre (o ENTER para saltar): 
   ⏩ Saltando a la siguiente parte...

🔹  Empiece conversaciones
      ❓ ¿Quién?
         Escribe Nombre (o ENTER para saltar): Dora
         [0] Dora Mendoza (F)
         👉 Elige #: 0
   💾 Registrado para guardar: Dora Mendoza
      ❓ ¿Qui